In [2]:
from getdata import CrossValidation
import matplotlib.pyplot as plt
import cv2
import numpy as np

cv = CrossValidation(as_image=False)
X, y, valid_x, valid_y = cv.get_set()

#binarize
X_c = np.array([[255 if x > 250 else 0 for x in y] for y in X])/255.0
valid_x = np.array([[255 if x > 250 else 0 for x in y] for y in valid_x])/255.0

beginning cross validation separation...
finished cross validation separation


In [ ]:
import matplotlib.pyplot as plt
plt.imshow(X_c[0].reshape((64,64)))

In [4]:
#baseline
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(hidden_layer_sizes=43, max_iter=1, alpha=1e-2,
                    solver='sgd', verbose=10, tol=1e-4, random_state=1,
                    learning_rate_init=.01, warm_start = True)
for i in range(100):
    clf.fit(X_c, y)
    if i % 10 == 0:
        print(clf.score(valid_x, valid_y))

Iteration 1, loss = 2.29225561
0.1555


/usr/local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Iteration 2, loss = 2.22297770
Iteration 3, loss = 2.16618011
Iteration 4, loss = 2.11518689
Iteration 5, loss = 2.06984257
Iteration 6, loss = 2.02756448
Iteration 7, loss = 1.98889708
Iteration 8, loss = 1.95263193
Iteration 9, loss = 1.91787931
Iteration 10, loss = 1.88345200
Iteration 11, loss = 1.84909281
0.2277
Iteration 12, loss = 1.81597277
Iteration 13, loss = 1.78413757
Iteration 14, loss = 1.75289003
Iteration 15, loss = 1.72241113
Iteration 16, loss = 1.69287809
Iteration 17, loss = 1.66311830
Iteration 18, loss = 1.63386990
Iteration 19, loss = 1.60518935
Iteration 20, loss = 1.57634331
Iteration 21, loss = 1.54832904
0.2176
Iteration 22, loss = 1.52023274
Iteration 23, loss = 1.49265397
Iteration 24, loss = 1.46520215
Iteration 25, loss = 1.43827649
Iteration 26, loss = 1.41187840
Iteration 27, loss = 1.38542230
Iteration 28, loss = 1.35918125
Iteration 29, loss = 1.33355076
Iteration 30, loss = 1.30810312
Iteration 31, loss = 1.28276903
0.2037
Iteration 32, loss = 1.2575

/usr/local/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:566: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Iteration 34, loss = 1.20174857
Iteration 35, loss = 1.17593527


KeyboardInterrupt: 

In [12]:
from scipy.special import expit, logit
from sklearn.metrics import f1_score

import sys
class NNClassifier:

    def __init__(self, n_classes, n_features, n_hidden_units=30,
                 epochs=500, lr=0.01, l2 = 0.001):
        np.random.seed(1)
        self.n_classes = n_classes
        self.n_features = n_features
        self.n_hidden_units = n_hidden_units
        self.w_hidden, self.w_out = self.weights()
        self.epochs = epochs
        self.learning_rate = lr
        self.l2 = l2

    def weights(self):
        w_hidden = np.random.uniform(-1.0, 1.0, size=self.n_hidden_units * (self.n_features + 1))
        w_hidden = w_hidden.reshape(self.n_hidden_units, self.n_features + 1)
        w_out = np.random.uniform(-1.0, 1.0, 
                               size=self.n_classes * (self.n_hidden_units + 1))
        w_out = w_out.reshape(self.n_classes, self.n_hidden_units + 1)
        return w_hidden, w_out

    def add_bias_unit(self, X, column=True):
        if column:
            bias_added = np.ones((X.shape[0], X.shape[1] + 1))
            bias_added[:, 1:] = X
        else:
            bias_added = np.ones((X.shape[0] + 1, X.shape[1]))
            bias_added[1:, :] = X

        return bias_added

    def _forward(self, X):
        nn_input = self.add_bias_unit(X, True)
        nn_hidden = self.w_hidden.dot(nn_input.T)
        act_hidden = self._sigmoid(nn_hidden)
        act_hidden = self.add_bias_unit(act_hidden, False)
        nn_out = self.w_out.dot(act_hidden)
        act_out = self._sigmoid(nn_out)
        return nn_input, nn_hidden, act_hidden, nn_out, act_out
    
    def _backward(self, nn_input, nn_hidden, act_hidden, act_out, y):
        s3 = act_out - y
        nn_hidden = self.add_bias_unit(nn_hidden, column=False)
        s2 = self.w_out.T.dot(s3) * self._sigmoid_prime(nn_hidden)
        s2 = s2[1:, :]
        g1 = s2.dot(nn_input)
        g2 = s3.dot(act_hidden.T)
        return g1, g2

    def _backpropogate(self, X, y):
        nn_input, nn_hidden, act_hidden, nn_out, act_out = self._forward(X)

        g1, g2 = self._backward(nn_input, nn_hidden, act_hidden, act_out, y)

        error = self._error(y.T, act_out)
        g1[:, 1:]+= (self.w_hidden[:, 1:]*self.l2) 
        g2[:, 1:]+= (self.w_out[:, 1:]*self.l2)
        return error, g1, g2
    
    def _cross_entropy(self, y, out):
        log_loss = - np.sum(y*np.log(out))
        return log_loss/y.shape[1]
    
    def _error(self, y, out):
        l2_term = (self.l2/2.0) * (np.sum(np.square(self.w_hidden[:, 1:])) + np.sum(np.square(self.w_out[:, 1:])))
        return self._cross_entropy(y, self._smax(out.T)) + l2_term
    
    def _sigmoid_prime(self, x):
        return self._sigmoid(x) * (1 - self._sigmoid(x)) 
    
    def _sigmoid(self, x):
        return expit(x)
    
    def _one_hot(self, y_data, n_classes):
        oh = np.zeros((len(y_data), n_classes))
        oh[np.arange(len(y_data)), y_data] = 1
        return oh
    
    def _smax(self, X):
        logC = -np.max(X)
        return np.exp(X + logC)/np.sum(np.exp(X + logC), axis = 0)
    
    def _mle(self, out):
        return [np.argmax(x) for x in self._smax(out)]
    
    def predict(self, X):
        X_c = X.copy()
        nn_input, nn_hidden, act_hidden, nn_out, act_out = self._forward(X_c)
        return self._mle(nn_out.T)
    
    def predict_proba(self, X):
        X_c = X.copy()
        nn_input, nn_hidden, act_hidden, nn_out, act_out = self._forward(X_c)
        return self._smax(act_out.T)

    def fit(self, X, y):
        self.error_ = []
        y_data_enc = self._one_hot(y, self.n_classes).T
        train_scores = []
        valid_scores = []
        for i in range(self.epochs):   
            # update weights
            error, g1, g2 = self._backpropogate(X, y_data_enc)
            self.error_.append(error)
            print('Iteration {}, loss = {}, valid_score=  {}'.format(i, error, self.score(valid_x, valid_y)))
            self.w_hidden -= (self.learning_rate * g1)
            self.w_out -= (self.learning_rate * g2)
        return self
    
    def score(self, X, y):
        y_hat = self.predict(X)
        return np.sum(y == y_hat, axis=0) / float(X.shape[0])

In [ ]:
n_hidden_units = np.arange(30, 40)
lr = [0.00001, 0.0005, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5]
l2 = np.arange(0.0001, 0.0009, 0.0001)
X_s = X_c[:1000]
y_s = y[:1000]
print(lr)
unit = np.random.choice(n_hidden_units)
learn_rate = np.random.choice(lr)
reg = np.random.choice(l2)
f1_history_lr = []
for i in lr:
    nn = NNClassifier(n_classes=10, 
                  n_features=4096,
                  n_hidden_units=40,
                  epochs=100,
                  lr=i, l2=0.001)
    print('[INFO]    Units: {}'.format(40))
    print('[INFO]    Learning Rate: {}'.format(i))
    print('[INFO]    L2 Regularization: {}'.format(reg))
    nn = nn.fit(X_s, y_s)
    f1_history_lr.append((nn, f1_score(valid_y, nn.predict(valid_x), average = 'macro')))
    


In [ ]:
import matplotlib.pyplot as plt
f1_scores_lr = [n[1] for n in f1_history_lr]
print(lr[np.argmax(f1_scores_lr)])
plt.plot(lr, f1_scores_lr)
print(np.ptp(f1_scores_lr))

In [ ]:
#Regularization Tuning

n_hidden_units = np.arange(30, 40)
l2 = [0.001, 0.005, 0.01, 0.05, 0.1, 1, 10, 100]
print(lr)
unit = np.random.choice(n_hidden_units)
learn_rate = np.random.choice(lr)
reg = np.random.choice(l2)
f1_history_l2 = []
for i in l2:
    nn = NNClassifier(n_classes=10, 
                  n_features=4096,
                  n_hidden_units=40,
                  epochs=100,
                  lr=lr[np.argmax(f1_scores_lr)], l2=i)
    print('[INFO]    Units: {}'.format(40))
    print('[INFO]    Learning Rate: {}'.format(lr[np.argmax(f1_scores_lr)], l2=i))
    print('[INFO]    L2 Regularization: {}'.format(i))
    nn = nn.fit(X_s, y_s)
    f1_history_l2.append((nn, f1_score(valid_y, nn.predict(valid_x), average = 'macro')))
    


In [ ]:
import matplotlib.pyplot as plt
f1_scores_l2 = [n[1] for n in f1_history_l2]
print(f1_scores_l2)
plt.plot(lr, f1_scores_l2)

In [ ]:
#units Tuning

n_hidden_units = np.arange(10, 70)
print(n_hidden_units)
f1_history_u = []
for i in n_hidden_units:
    nn = NNClassifier(n_classes=10, 
                  n_features=4096,
                  n_hidden_units=i,
                  epochs=100,
                  lr=lr[np.argmax(f1_scores_lr)], l2=l2[np.argmax(f1_scores_l2)])
    print('[INFO]    Units: {}'.format(i))
    print('[INFO]    Learning Rate: {}'.format(lr[np.argmax(f1_scores_lr)], l2=i))
    print('[INFO]    L2 Regularization: {}'.format(l2[np.argmax(f1_scores_l2)]))
    nn = nn.fit(X_s, y_s)
    f1_history_u.append((nn, f1_score(valid_y, nn.predict(valid_x), average = 'macro')))
    


In [ ]:
import matplotlib.pyplot as plt
f1_scores_u = [n[1] for n in f1_history_u]
print(f1_scores_u)
plt.plot(n_hidden_units, f1_scores_u)
print('\n{}'.format(n_hidden_units[np.argmax(f1_scores_u)]))
print(np.amax(f1_scores_u))
print(np.ptp(f1_scores_u))

In [ ]:
nn = NNClassifier(n_classes=10, 
                  n_features=4096,
                  n_hidden_units=20,
                  epochs=1000,
                  lr=0.0001, l2=1e2)
nn = nn.fit(X_c, y)

Iteration 0, loss = 1416726.2484588146, valid_score=  0.1009
Iteration 1, loss = 1395697.7516761625, valid_score=  0.1043
Iteration 2, loss = 1366728.9141277857, valid_score=  0.1061
Iteration 3, loss = 1339541.3130826913, valid_score=  0.1037
Iteration 4, loss = 1313308.8741692393, valid_score=  0.1037
Iteration 5, loss = 1287750.5342154412, valid_score=  0.1038
Iteration 6, loss = 1262759.5594741402, valid_score=  0.1038
Iteration 7, loss = 1238295.29136828, valid_score=  0.1064
Iteration 8, loss = 1214338.0968658552, valid_score=  0.1052
Iteration 9, loss = 1190873.9534195492, valid_score=  0.1055
Iteration 10, loss = 1167890.7291667056, valid_score=  0.1063
Iteration 11, loss = 1145377.181654912, valid_score=  0.1059
Iteration 12, loss = 1123322.6754312562, valid_score=  0.1066
Iteration 13, loss = 1101716.900827746, valid_score=  0.1073
Iteration 14, loss = 1080549.8733123483, valid_score=  0.1087
Iteration 15, loss = 1059811.9443021212, valid_score=  0.1092
Iteration 16, loss = 1

Iteration 134, loss = 135612.53094737022, valid_score=  0.1643
Iteration 135, loss = 133770.03043216994, valid_score=  0.1641
Iteration 136, loss = 131964.42352840205, valid_score=  0.1636
Iteration 137, loss = 130194.97734461854, valid_score=  0.1628
Iteration 138, loss = 128460.97356774267, valid_score=  0.163
Iteration 139, loss = 126761.70817112508, valid_score=  0.1631
Iteration 140, loss = 125096.49112754705, valid_score=  0.1635
Iteration 141, loss = 123464.64612719518, valid_score=  0.1644
Iteration 142, loss = 121865.51030061906, valid_score=  0.1648
Iteration 143, loss = 120298.43394667497, valid_score=  0.1649
Iteration 144, loss = 118762.78026545588, valid_score=  0.1659
Iteration 145, loss = 117257.9250962142, valid_score=  0.1662
Iteration 146, loss = 115783.2566602983, valid_score=  0.166
Iteration 147, loss = 114338.17530914024, valid_score=  0.1674
Iteration 148, loss = 112922.09327734614, valid_score=  0.1685
Iteration 149, loss = 111534.43444094872, valid_score=  0.1

Iteration 267, loss = 50116.26003580007, valid_score=  0.1823
Iteration 268, loss = 49943.41815918022, valid_score=  0.1908
Iteration 269, loss = 49891.788844791416, valid_score=  0.1808
Iteration 270, loss = 49712.45484360485, valid_score=  0.1881
Iteration 271, loss = 49644.64368948562, valid_score=  0.1837
Iteration 272, loss = 49481.46615356227, valid_score=  0.1913
Iteration 273, loss = 49438.219365359706, valid_score=  0.1824
Iteration 274, loss = 49269.11430557536, valid_score=  0.189
Iteration 275, loss = 49210.802984004884, valid_score=  0.1841
Iteration 276, loss = 49056.600338377444, valid_score=  0.1913
Iteration 277, loss = 49020.94621687973, valid_score=  0.1836
Iteration 278, loss = 48861.40241028878, valid_score=  0.1894
Iteration 279, loss = 48811.82343414999, valid_score=  0.1847
Iteration 280, loss = 48665.96777752352, valid_score=  0.1914
Iteration 281, loss = 48637.16410381057, valid_score=  0.1842
Iteration 282, loss = 48486.57866183414, valid_score=  0.1909
Itera

Iteration 399, loss = 44859.05588175332, valid_score=  0.1861
Iteration 400, loss = 44787.494711524654, valid_score=  0.1936
Iteration 401, loss = 44839.50099564972, valid_score=  0.1827
Iteration 402, loss = 44775.6206307829, valid_score=  0.1967
Iteration 403, loss = 44793.76955328476, valid_score=  0.1923
Iteration 404, loss = 44751.939163227915, valid_score=  0.1978
Iteration 405, loss = 44801.5323592962, valid_score=  0.1876
Iteration 406, loss = 44746.39185450841, valid_score=  0.1949
Iteration 407, loss = 44793.181571910594, valid_score=  0.1875
Iteration 408, loss = 44736.53609689108, valid_score=  0.1935
Iteration 409, loss = 44793.56218433655, valid_score=  0.1849
Iteration 410, loss = 44734.60720980097, valid_score=  0.1942
Iteration 411, loss = 44762.95842636683, valid_score=  0.1929
Iteration 412, loss = 44718.677313664346, valid_score=  0.1948
Iteration 413, loss = 44778.65596979632, valid_score=  0.188
Iteration 414, loss = 44722.56337356126, valid_score=  0.194
Iteratio

Iteration 532, loss = 44791.74254326699, valid_score=  0.2016
Iteration 533, loss = 44830.01788682776, valid_score=  0.1939
Iteration 534, loss = 44789.3077104863, valid_score=  0.1993
Iteration 535, loss = 44861.019703746584, valid_score=  0.1913
Iteration 536, loss = 44808.18028057802, valid_score=  0.2021
Iteration 537, loss = 44835.54671222891, valid_score=  0.1943
Iteration 538, loss = 44800.05353383182, valid_score=  0.202
Iteration 539, loss = 44878.63356587528, valid_score=  0.1917
Iteration 540, loss = 44823.65182584604, valid_score=  0.2036
Iteration 541, loss = 44843.53733188167, valid_score=  0.1941
Iteration 542, loss = 44812.39194280773, valid_score=  0.2024
Iteration 543, loss = 44884.49385169869, valid_score=  0.1887
Iteration 544, loss = 44835.10702959579, valid_score=  0.1971
Iteration 545, loss = 44850.77952747655, valid_score=  0.1958
Iteration 546, loss = 44815.382132753555, valid_score=  0.2007
Iteration 547, loss = 44876.09130459317, valid_score=  0.1942
Iteratio

Iteration 664, loss = 45071.53055801835, valid_score=  0.2021
Iteration 665, loss = 45134.21766440306, valid_score=  0.1955
Iteration 666, loss = 45081.57004129204, valid_score=  0.2024
Iteration 667, loss = 45123.6678302908, valid_score=  0.1964
Iteration 668, loss = 45080.579931882625, valid_score=  0.2031
Iteration 669, loss = 45152.107789004775, valid_score=  0.1947
Iteration 670, loss = 45096.679359358815, valid_score=  0.2029
Iteration 671, loss = 45130.71840416577, valid_score=  0.1953
Iteration 672, loss = 45091.22114286039, valid_score=  0.2032
Iteration 673, loss = 45175.97473229763, valid_score=  0.1913
Iteration 674, loss = 45115.37658444532, valid_score=  0.2016
Iteration 675, loss = 45129.48067961891, valid_score=  0.1971
Iteration 676, loss = 45103.77370664811, valid_score=  0.1974
Iteration 677, loss = 45160.70962638192, valid_score=  0.1944
Iteration 678, loss = 45104.902381264146, valid_score=  0.1994
Iteration 679, loss = 45142.15428541794, valid_score=  0.1957
Itera

In [21]:
x = np.loadtxt("test_x.csv", delimiter=",")

0.2172

In [22]:
test_x = np.loadtxt("../input/test_x.csv", delimiter=",")

In [24]:
test_x_c = np.array([[255 if x > 250 else 0 for x in y] for y in test_x])/255.0

In [25]:
predictions = nn.predict(test_x_c)
import pandas as pd
dt = pd.DataFrame(data=predictions)
dt.columns = ["Label"]
dt = dt.astype(int)
dt.to_csv('../output/nn_predictions.csv', mode='w',index=True, index_label='Id')

In [26]:
predictions


[1,
 5,
 3,
 8,
 6,
 1,
 6,
 0,
 8,
 9,
 3,
 1,
 1,
 4,
 8,
 4,
 3,
 4,
 0,
 7,
 8,
 0,
 0,
 1,
 0,
 0,
 9,
 8,
 6,
 0,
 1,
 3,
 7,
 6,
 4,
 3,
 0,
 5,
 0,
 4,
 8,
 6,
 0,
 1,
 0,
 6,
 2,
 1,
 1,
 6,
 1,
 7,
 3,
 1,
 6,
 6,
 2,
 0,
 3,
 0,
 1,
 8,
 3,
 6,
 1,
 1,
 6,
 1,
 1,
 4,
 3,
 7,
 8,
 0,
 0,
 1,
 1,
 7,
 4,
 1,
 4,
 4,
 1,
 1,
 8,
 6,
 3,
 7,
 1,
 4,
 6,
 3,
 0,
 7,
 1,
 3,
 4,
 1,
 7,
 5,
 7,
 8,
 3,
 2,
 6,
 7,
 0,
 1,
 6,
 2,
 6,
 7,
 8,
 8,
 4,
 0,
 1,
 1,
 0,
 3,
 1,
 6,
 4,
 0,
 4,
 6,
 6,
 8,
 4,
 7,
 1,
 3,
 5,
 4,
 2,
 7,
 0,
 3,
 8,
 7,
 1,
 7,
 4,
 3,
 5,
 1,
 2,
 0,
 4,
 0,
 3,
 3,
 3,
 1,
 1,
 1,
 8,
 3,
 3,
 0,
 3,
 4,
 0,
 7,
 4,
 0,
 1,
 4,
 0,
 2,
 6,
 9,
 8,
 4,
 9,
 0,
 3,
 1,
 9,
 7,
 3,
 7,
 5,
 0,
 3,
 0,
 6,
 0,
 1,
 5,
 6,
 0,
 4,
 1,
 8,
 8,
 6,
 7,
 3,
 1,
 7,
 4,
 0,
 8,
 3,
 3,
 9,
 1,
 2,
 2,
 7,
 6,
 7,
 2,
 2,
 1,
 7,
 7,
 9,
 8,
 8,
 1,
 7,
 3,
 1,
 0,
 7,
 7,
 7,
 8,
 4,
 7,
 4,
 2,
 6,
 1,
 7,
 1,
 0,
 6,
 8,
 5,
 6,
 7,
 9,
 3,
 0,
 1,
 0,
 4,
